In [1]:
import jupyanno as ja
import numpy as np

In [2]:
from traitlets import Unicode, validate, Bytes, Int as TrInt
import ipywidgets as widgets
class CornerstoneWidget(widgets.DOMWidget):
    _view_name = Unicode('CornerstoneWidget').tag(sync=True)
    _view_module = Unicode('cs_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    title_field = Unicode('Awesome Widget').tag(sync=True)
    img_bytes = Unicode('AQAAAAAAAAABAAAAAAAAAAEA').tag(sync=True)
    img_width = TrInt(3).tag(sync=True)
    img_height= TrInt(3).tag(sync=True)

In [3]:
%%javascript
require.config({
  paths: {
      'cornerstone-core': '//unpkg.com/cornerstone-core@2.2.4/dist/cornerstone.min',
      cornerstoneMath: '//unpkg.com/cornerstone-math@0.1.6/dist/cornerstoneMath.min',
      cornerstoneTools: '//unpkg.com/cornerstone-tools@2.3.9/dist/cornerstoneTools.min'
  },
    shim: {
        'cornerstone-core': {
            exports: 'cornerstone',
            deps: ['jquery']
        }
    }
});

<IPython.core.display.Javascript object>

In [4]:
%%javascript
require.undef('cs_widget');
define('cs_widget', ["@jupyter-widgets/base", "cornerstone-core",
                    'cornerstoneMath', 'cornerstoneTools'], 
       function(widgets, cs, cm, ctools) {
            ctools.external.cornerstone = cs;
            ctools.external.cornerstoneMath = cm;
    var CornerstoneWidget = widgets.DOMWidgetView.extend({

        render: function() {
            this.message = document.createElement('div')
            this.viewer = document.createElement('div')
            var fv = $(this.viewer)
            fv.width('512px');
            fv.height('512px');
            // Enable our tools
            this.el.appendChild(this.message);
            this.el.appendChild(this.viewer);
            this.model.on('change:img_bytes', this.dicom_changed, this);
            this.model.on('change:title_field', this.message_change, this);
        },
        
        parse_image: function(imageB64Data, width, height) {
            function str2ab(str) {
                var buf = new ArrayBuffer(str.length*2); // 2 bytes for each char
                var bufView = new Uint16Array(buf);
                var index = 0;
                for (var i=0, strLen=str.length; i<strLen; i+=2) {
                    var lower = str.charCodeAt(i);
                    var upper = str.charCodeAt(i+1);
                    bufView[index] = lower + (upper <<8);
                    index++;
                }
                return bufView;
            }
            function parsePixelData(base64PixelData, width, height)
            {
                var pixelDataAsString = window.atob(base64PixelData);
                var pixelData = str2ab(pixelDataAsString);
                return pixelData;
            }
            var imagePixelData = parsePixelData(imageB64Data);
            console.log('decoding: '+width+'x'+height+' => '+imagePixelData.length)
            function getPixelData() {
                return imagePixelData;
            }
            return {
                imageId: 'imageId',
                minPixelValue: 0.0,
                maxPixelValue: 255.0,
                slope: 1.0,
                intercept: 0.0,
                windowCenter : 127,
                windowWidth : 127,
                getPixelData: getPixelData,
                rows: width,
                columns: height,
                height: height,
                width: width,
                color: false,
                columnPixelSpacing: .8984375,
                rowPixelSpacing: .8984375,
                sizeInBytes: height * width * 2
            };
        },
        message_change: function() {
            this.message.textContent = this.model.get('title_field');
        },
        dicom_changed: function() {
            var img_bytes = this.model.get('img_bytes')
            var img_width = this.model.get('img_width')
            var img_height = this.model.get('img_height')
            var out_img = this.parse_image(img_bytes, img_width, img_height);
            cs.enable(this.viewer);
            var viewport = cs.getDefaultViewportForImage(this.viewer, out_img);
            console.log(out_img);
            cs.displayImage(this.viewer, out_img, viewport);
            ctools.mouseInput.enable(this.viewer);
            ctools.mouseWheelInput.enable(this.viewer);
            ctools.wwwc.activate(this.viewer, 1); // Left Click
            ctools.pan.activate(this.viewer, 2); // Middle Click
            ctools.zoom.activate(this.viewer, 4); // Right Click
            ctools.zoomWheel.activate(this.viewer); // Mouse Wheel
        },
    });

    return {
        CornerstoneWidget : CornerstoneWidget
    };
});

<IPython.core.display.Javascript object>

In [5]:
w = CornerstoneWidget()
w

CornerstoneWidget()

In [6]:
w.title_field = 'Cornerstone Widget'

In [7]:
out_array = 255*np.eye(8)
(w.img_width, w.img_height) = out_array.shape
w.img_bytes  = ja.cornerstone.encode_numpy_b64(out_array)